Этот ноутбук должен был стать основным решением, но у меня не получилось его завести. Попрошу все-таки дать ему шанс и посмотреть полностью.

Библеотека
```
adapter-transformers
```
Запустилась успешно 1 раз, в ходе которого и был написан и проверен код и он работал.

Дальше отдельные элементы упорно не хотели грузится, как бы я не пытался их завести.

Более того выгруженная модель оказалась битой и я не смог её никуда подключить.

Единственное, что осталось - это скриншоты с не совсем уместными примерами работы classifier

In [ ]:
#устанавливаем библеотеки
#!pip uninstall transformers
!pip install -U adapter-transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.19.4
    Uninstalling huggingface-hub-0.19.4:
      Successfully uninstalled huggingface-hub-0.19.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.16.0 requires huggingface-hub>=0.19.4, but you have huggingface-hub 0.13.4 which is incompatible.
transformers 4.35.2 requires huggingface-hub<1.0,>=0.16.4, but you have huggingface-hub 0.13.4 which is incompati

In [ ]:
#подгружаем библеотеки
import datasets
import pandas as pd
from transformers import RobertaConfig, EvalPrediction, TrainingArguments,  RobertaTokenizer, TextClassificationPipeline, AutoModelForSequenceClassification
#from adapters import AdapterTrainer
import json
import torch
from typing import Tuple
from datasets import load_dataset
#import git
import os
import platform

In [ ]:
#специально для колаба загрузка данных с лемматизированными жалобами
'''
from google.colab import files
files.upload()
!ls
'''

Saving Mtex_lemmed to Mtex_lemmed
Mtex_lemmed  sample_data


In [ ]:
df = pd.read_csv('Mtex_lemmed', index_col = 0)

In [ ]:
df

,id,text,class,lemm
0,45,негатив к вам и горячей линии которые нафиг н...,Консультация КЦ,негатив горячий линия который нафиг нужный...
1,97,перестаньте впаривать клиентам дополнительные ...,Компетентность продавцов/ консультантов,переставать впаривать клиент дополнительный то...
2,105,продавец предложил наклеить защитную плёнку з...,Компетентность продавцов/ консультантов,продавец предлагать наклеивать защитный плнка ...
3,127,продавец режил продать нам товар почему то по ...,Компетентность продавцов/ консультантов,продавец реживать продавать мы товар почему ...
4,130,навязанные услуги ( без моего согласия) на сум...,Компетентность продавцов/ консультантов,навязывать услуга сумма рубль\n
...,...,...,...,...
2832,17570,почему отключили электронную очередь через пол...,Электронная очередь,почему отключать электронный очередь получени...
2833,17579,очередь на кассе. продавцы не торопятся с зака...,Время ожидания у кассы,очередь касса продавец торопиться заказ ра...
2834,17586,"профессионализма менеджеру бы, да и внимания к...",Обслуживание продавцами/ консультантами,профессионализм менеджер внимание клиент м...
2835,17591,сотрудник на выдаче заказа потребовал документ...,Электронная очередь,сотрудник выдача заказ потребовать документ у...


In [ ]:
#добавление лейблов для dataset
labels = dict(enumerate(map(str, df['class'].unique())))

labels_inv = {value: key for key, value in labels.items()}
df['labels'] = df['class'].map(labels_inv)
df

,id,text,class,lemm,labels
0,45,негатив к вам и горячей линии которые нафиг н...,Консультация КЦ,негатив горячий линия который нафиг нужный...,0
1,97,перестаньте впаривать клиентам дополнительные ...,Компетентность продавцов/ консультантов,переставать впаривать клиент дополнительный то...,1
2,105,продавец предложил наклеить защитную плёнку з...,Компетентность продавцов/ консультантов,продавец предлагать наклеивать защитный плнка ...,1
3,127,продавец режил продать нам товар почему то по ...,Компетентность продавцов/ консультантов,продавец реживать продавать мы товар почему ...,1
4,130,навязанные услуги ( без моего согласия) на сум...,Компетентность продавцов/ консультантов,навязывать услуга сумма рубль\n,1
...,...,...,...,...,...
2832,17570,почему отключили электронную очередь через пол...,Электронная очередь,почему отключать электронный очередь получени...,2
2833,17579,очередь на кассе. продавцы не торопятся с зака...,Время ожидания у кассы,очередь касса продавец торопиться заказ ра...,7
2834,17586,"профессионализма менеджеру бы, да и внимания к...",Обслуживание продавцами/ консультантами,профессионализм менеджер внимание клиент м...,6
2835,17591,сотрудник на выдаче заказа потребовал документ...,Электронная очередь,сотрудник выдача заказ потребовать документ у...,2


In [ ]:
#подгрузка ruRoberta-large
from transformers import RobertaTokenizer
MODEL_NAME = 'ai-forever/ruRoberta-large'
tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/1.81M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

In [ ]:
df_train = df[['text', 'labels']]

In [ ]:
#Разбивка на обучающую и тестовую выборки
from sklearn.model_selection import train_test_split

train_size = 0.8  # 80% для тренировочного датасета
test_size = 0.2  # 20% для тестового датасета

train_set, test_set = train_test_split(df_train, train_size=train_size, test_size=test_size, random_state=42)

print("Размер тренировочного датасета:", len(train_set))
print("Размер тестового датасета:", len(test_set))

Размер тренировочного датасета: 2269
Размер тестового датасета: 568


In [ ]:
train_set.to_csv('m_tex_for_llm_train.csv')
test_set.to_csv('m_tex_for_llm_test.csv')

In [ ]:
#Из pandas в hugging-face dataset
dataset = load_dataset('csv', data_files={'train': ['m_tex_for_llm_train.csv'], 'test': ['m_tex_for_llm_test.csv']})

NameError: ignored

In [ ]:
def encode_batch(batch):
  """Кодируем батч входных данных с помощью токенизатора модели."""
  print(batch.keys())
  return tokenizer(batch["text"], max_length=128, truncation=True, padding="max_length")

In [ ]:
dataset = dataset.map(encode_batch, batched=True)
# Преобразование в тензоры pytorch и вывод только нужных столбцов
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/2269 [00:00<?, ? examples/s]

KeysView({'Unnamed: 0': [940, 741, 2798, 2204, 787, 1592, 1088, 1102, 651, 2270, 461, 1456, 157, 1720, 1508, 700, 238, 2580, 1380, 1777, 100, 315, 807, 829, 612, 2678, 261, 729, 2801, 1883, 1945, 179, 1709, 1786, 1446, 2688, 889, 480, 1349, 2304, 1844, 1366, 1876, 2284, 1530, 746, 1614, 1718, 2617, 2809, 247, 2533, 1911, 347, 842, 1600, 2597, 2833, 2484, 1378, 2003, 445, 617, 1100, 1134, 979, 1764, 1412, 2414, 547, 1591, 263, 1124, 2108, 353, 208, 2766, 1375, 1889, 1572, 1357, 57, 2213, 2541, 561, 229, 2342, 2286, 817, 2593, 900, 2720, 998, 424, 69, 2131, 2542, 2821, 1068, 1196, 620, 679, 1080, 2208, 1564, 1340, 2746, 2337, 2226, 279, 759, 786, 2243, 1809, 1904, 2632, 596, 1419, 2008, 2574, 440, 508, 76, 1712, 649, 522, 2611, 1651, 1350, 2479, 1391, 1677, 1487, 630, 2648, 2686, 858, 482, 2058, 1321, 2425, 1364, 438, 1005, 1393, 1784, 166, 59, 611, 530, 1532, 49, 1337, 803, 1094, 48, 719, 2312, 2299, 2123, 1710, 632, 1298, 999, 2614, 25, 1192, 2404, 276, 2039, 2477, 2012, 361, 1611, 139

Map:   0%|          | 0/568 [00:00<?, ? examples/s]

KeysView({'Unnamed: 0': [415, 1596, 1322, 2271, 1966, 354, 965, 1599, 527, 1770, 1642, 2114, 2599, 324, 677, 2424, 1817, 794, 1362, 402, 937, 1991, 532, 1177, 811, 2726, 2328, 2740, 533, 1804, 841, 1656, 2162, 748, 1513, 765, 141, 2742, 1204, 2567, 1717, 1398, 2808, 1113, 944, 819, 1556, 2579, 1788, 387, 495, 432, 2527, 927, 2810, 802, 2104, 2087, 1093, 1422, 2773, 430, 602, 296, 2350, 1431, 2438, 1543, 544, 1518, 1557, 2190, 2578, 1048, 194, 2780, 2396, 2313, 1497, 581, 2234, 929, 767, 1747, 1278, 1867, 1729, 2703, 32, 2355, 67, 2102, 1476, 1230, 1567, 2698, 1754, 1939, 298, 2309, 941, 840, 1029, 318, 1429, 2180, 2440, 2491, 2323, 1407, 309, 2555, 498, 1948, 1127, 2823, 1929, 2634, 196, 1292, 2246, 862, 1814, 644, 30, 439, 1027, 2677, 568, 1580, 1272, 1561, 178, 859, 1533, 1732, 2372, 2831, 2406, 1791, 2394, 2225, 73, 1200, 63, 1618, 211, 45, 1389, 2183, 1174, 1450, 80, 2473, 2711, 1103, 976, 479, 2429, 1410, 1381, 408, 1601, 2523, 1465, 691, 2668, 1488, 2126, 2242, 2767, 599, 1628, 1

In [ ]:
from transformers import RobertaModelWithHeads

In [ ]:
from transformers import RobertaTokenizer, RobertaConfig
config = RobertaConfig.from_pretrained(
    MODEL_NAME,
    num_labels=2,
)
model = RobertaModelWithHeads.from_pretrained(
    MODEL_NAME,
    config=config,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/adapters/models/roberta/adapter_model.py:274: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/adapters/models/roberta/adapter_model.py:252: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
Some weights of the model checkpoint at ai-forever/ruRoberta-large were not used when initializing RobertaModelWithHeads: ['lm_head.decoder.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model f

In [ ]:
#создаем новый адаптер
model.add_adapter("mtex")
model.add_classification_head(

    "complaint",
    num_labels=len(labels),
    id2label=labels
  )

In [ ]:
#заморозим все веса в модели, кроме веса адаптера.
model.train_adapter("mtex")

In [ ]:
from transformers import AdapterTrainer

RuntimeError: ignored

In [ ]:
#прописываю параметры обучения
import numpy as np
from transformers import TrainingArguments, Trainer, EvalPrediction
training_args = TrainingArguments(
    learning_rate=5e-5,
    num_train_epochs=20,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    logging_steps=20,
    output_dir="./training_output",
    overwrite_output_dir=False,
    remove_unused_columns=True,

)

# def compute_accuracy(p: EvalPrediction):
#     preds = np.argmax(p.predictions, axis=1)
#     return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
     #compute_metrics=EvalPrediction,
)

RuntimeError: ignored

In [ ]:
#обучаю адаптер
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
from transformers import TextClassificationPipeline
#удобный пайплайн для тестирования
classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=training_args.device.index)

In [ ]:
classifier('продавец наорал мне в лицо')

In [ ]:
classifier('стоял в очереди 40 часов, умер от обезвоживания')

In [ ]:
classifier.save_pretrained('/content/drive/MyDrive/MtexModel/')

In [ ]:
classifier('пытался выбрать фен, за пол часа никто не подошел')